# Using `ROMS-Tools` with `Dask`

This notebook aims to:
* Explain how to use `ROMS-Tools` with `Dask`
* Identify when `Dask` is beneficial for `ROMS-Tools` workflows
* Demonstrate different `Dask` deployment options
* Provide profiling insights for performance optimization

## How to use `ROMS-Tools` with `Dask`

[Dask](https://www.dask.org/) is a Python parallel computing library that enables scalable, out-of-memory data processing by distributing computations across multiple threads or processes. 
`ROMS-Tools` integrates with `Dask` for several key classes where handling large datasets efficiently is essential:
1. tidal forcing: `TidalForcing`
2. surface forcing: `SurfaceForcing`
3. boundary forcing: `BoundaryForcing`
4. initial conditions: `InitialConditions`
5. ROMS output: `ROMSOutput`

Each of these classes can be used with or without `Dask`, controlled by the `use_dask` parameter. By default, `use_dask` is set to `False`, primarily because `Dask` is an optional dependency, and not all users may have it installed. However, enabling `use_dask = True` is essential for preventing memory issues and speeding up computations when working with large datasets. We will discuss *when* to use `Dask` in [this section](#when-to-use-dask) below, but first, let's go over *how* to use `Dask` with `ROMS-Tools`.

Let's explore an example using the following grid:

In [1]:
from roms_tools import Grid, InitialConditions
from datetime import datetime

In [2]:
grid = Grid(nx=500, ny=500, size_x=1000, size_y=1000, center_lon=-20, center_lat=65, rot=0)

In the following example, we set up initial conditions (with BGC) using the `InitialConditions` class, first without `Dask` and then with `Dask`. The resulting data will be identical in both cases—the key difference lies in how computations are executed. 

Let's work with the following fixed parameters:

In [3]:
initial_conditions_parameters = {
    "grid": grid,
    "ini_time": datetime(2012, 1, 2),
    "source": {"name": "GLORYS", "path":  "/glade/derecho/scratch/noraloose/Datasets/GLORYS/NA/2012/mercatorglorys12v1_gl12_mean_20120102.nc"},
    "bgc_source": {"name": "CESM_REGRIDDED", "path": "/glade/derecho/scratch/noraloose/Datasets/CESM_REGRIDDED/CESM-climatology_lowres_regridded.nc", "climatology": True}
}

In [4]:
%load_ext memory_profiler

### Without Dask (`use_dask = False`)
* Computations run serially (not parallelized).
* Computations occur during class initialization, meaning object creation takes longer.
* The `.ds` attribute holds eagerly evaluated data.
* The `.plot()` and `.save()` methods execute quickly since the data is already computed.

In [ ]:
%%time 
%%memit

initial_conditions_without_dask = InitialConditions(
    **initial_conditions_parameters,
    use_dask=False
)

INFO - Sea surface height will NOT be used to adjust depth coordinates.
INFO - Selected time entry closest to the specified start_time (2012-01-02 00:00:00) within the range [2012-01-02 00:00:00, 2012-01-03 00:00:00]: ['2012-01-02T12:00:00.000000000']
Process MemTimer-1:
Traceback (most recent call last):
  File "/glade/work/noraloose/conda-envs/romstools-test/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/glade/work/noraloose/conda-envs/romstools-test/lib/python3.12/site-packages/memory_profiler.py", line 262, in run
    stop = self.pipe.poll(self.interval)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/glade/work/noraloose/conda-envs/romstools-test/lib/python3.12/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
           ^^^^^^^^^^^^^^^^^^^
  File "/glade/work/noraloose/conda-envs/romstools-test/lib/python3.12/multiprocessing/connection.py", line 440, in _poll
    r = wait([self], timeout)
        ^^^^^^

As seen in the summary above, initialization took over 20 minutes and consumed ~56GB of memory (this was on a compute node on [Casper](https://ncar-hpc-docs.readthedocs.io/en/latest/compute-systems/casper/)). The long compute times and high memory footprint occur because all computations were performed during initialization.

We can confirm this by inspecting the `.ds` attribute, which holds an xarray Dataset. As expected, all data arrays are eagerly evaluated, meaning they are fully loaded into memory rather than being computed lazily.

In [ ]:
initial_conditions_without_dask.ds

Since the data has already been computed, both plotting and saving are nearly instantaneous and require minimal additional memory. The next two cells demonstrate this.

In [ ]:
%%time 
%%memit

initial_conditions_without_dask.plot("u", s=-1)

In [ ]:
filename = "/glade/derecho/scratch/noraloose/forcing/my_initial_conditions.nc"

In [ ]:
%%time 
%%memit

initial_conditions_without_dask.save(filename)

### With Dask (`use_dask = True`)
* Computations are parallelized using `Dask`.
* Computations are delayed until `.plot()` or `.save()` is called (except for a few variables used for validation), meaning object creation is fast.
* The `.ds` attribute holds lazy (unevaluated) data.
* The `.plot()` and `.save()` methods take longer to run, as computations occur at this stage.

In [5]:
%%time 
%%memit

initial_conditions_with_dask = InitialConditions(
    **initial_conditions_parameters,
    use_dask=True
)

INFO - Sea surface height will NOT be used to adjust depth coordinates.
INFO - Selected time entry closest to the specified start_time (2012-01-02 00:00:00) within the range [2012-01-02 00:00:00, 2012-01-03 00:00:00]: ['2012-01-02T12:00:00.000000000']


peak memory: 4159.71 MiB, increment: 3876.12 MiB
CPU times: user 56.2 s, sys: 25.5 s, total: 1min 21s
Wall time: 15 s


As seen in the summary above, initialization was significantly faster and consumed far less memory compared to `use_dask = False`. This is because computations are deferred until explicitly triggered, rather than being performed immediately upon initialization. 

As a result, the `.ds` attribute holds an xarray Dataset with lazy data arrays, meaning the actual computations occur only when needed, such as during plotting or saving. An exception to this are the variables `w`, `Cs_r`, `Cs_w`, and `zeta`:
* `w` is set to zero and stored eagerly.
* `Cs_r` and `Cs_w` are copied from `grid.ds`, where they were originally stored eagerly.
* `zeta` is computed eagerly to check for NaNs in the interpolated field during a validation step. We will explore the validation step further in the #bypassing-validation section later on.

In [6]:
initial_conditions_with_dask.ds

<xarray.Dataset> Size: 4GB
Dimensions:      (ocean_time: 1, eta_rho: 502, xi_rho: 502, s_rho: 100,
                  xi_u: 501, eta_v: 501, s_w: 101)
Coordinates:
    abs_time     (ocean_time) datetime64[ns] 8B 2012-01-02T12:00:00
  * ocean_time   (ocean_time) float64 8B 3.788e+08
Dimensions without coordinates: eta_rho, xi_rho, s_rho, xi_u, eta_v, s_w
Data variables: (12/42)
    zeta         (ocean_time, eta_rho, xi_rho) float32 1MB -0.7429 ... -0.8718
    temp         (ocean_time, s_rho, eta_rho, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    salt         (ocean_time, s_rho, eta_rho, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    u            (ocean_time, s_rho, eta_rho, xi_u) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    v            (ocean_time, s_rho, eta_v, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    ubar         (ocean_time, eta_rho, xi_u) float32 1MB dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    ...           ...
    diazFe       (ocean_time, s_rho, eta_rho, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    spCaCO3      (ocean_time, s_rho, eta_rho, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    zooC         (ocean_time, s_rho, eta_rho, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    w            (ocean_time, s_w, eta_rho, xi_rho) float32 102MB 0.0 ... 0.0
    Cs_r         (s_rho) float32 400B -0.992 -0.9753 ... -8.89e-05 -9.874e-06
    Cs_w         (s_w) float32 404B -1.0 -0.9837 -0.9667 ... -3.95e-05 0.0
Attributes:
    title:                                ROMS initial conditions file create...
    roms_tools_version:                   0.1.dev165
    ini_time:                             2012-01-02 00:00:00
    model_reference_date:                 2000-01-01 00:00:00
    adjust_depth_for_sea_surface_height:  False
    source:                               GLORYS
    bgc_source:                           CESM_REGRIDDED
    theta_s:                              5.0
    theta_b:                              2.0
    hc:                                   300.0

Plotting triggers the computation of the requested variable, which results in non-negligible compute time and memory usage. 

In [ ]:
%%time 
%%memit

initial_conditions_with_dask.plot("u", s=-1)

[########                                ] | 21% Completed | 2.20 sms

Process MemTimer-2:


[########                                ] | 21% Completed | 2.30 s


Traceback (most recent call last):
  File "/glade/work/noraloose/conda-envs/romstools-test/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/glade/work/noraloose/conda-envs/romstools-test/lib/python3.12/site-packages/memory_profiler.py", line 262, in run
    stop = self.pipe.poll(self.interval)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/glade/work/noraloose/conda-envs/romstools-test/lib/python3.12/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
           ^^^^^^^^^^^^^^^^^^^
  File "/glade/work/noraloose/conda-envs/romstools-test/lib/python3.12/multiprocessing/connection.py", line 440, in _poll
    r = wait([self], timeout)
        ^^^^^^^^^^^^^^^^^^^^^
  File "/glade/work/noraloose/conda-envs/romstools-test/lib/python3.12/multiprocessing/connection.py", line 1136, in wait
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/glade/work/noraloose/conda-envs/romstools-test/lib/

Saving triggers the computation of the entire dataset. However, thanks to parallelization, it is still faster and more memory-efficient compared to the initialization step when using `use_dask = False`!

In [ ]:
%%time 
%%memit

initial_conditions_with_dask.save(filename)

Finally, we confirm that the two datasets that were created are exactly the same.

In [14]:
initial_conditions_without_dask.ds.identical(initial_conditions_with_dask.ds)

False

### Internal "chunking" within `ROMS-Tools`

In `Dask`, chunking refers to the process of dividing a large dataset into **smaller, manageable pieces (chunks)**, allowing parallel computation across those pieces **without needing to load the entire dataset into memory at once**.

For the classes `TidalForcing`, `SurfaceForcing`, `BoundaryForcing`, and `ROMSOutput`, the `use_dask` parameter is the primary setting for `Dask` configuration during class initialization. No further `Dask`-related parameters are needed because `Dask` **chunking is managed internally** in `ROMS-Tools` based on logical choices:

* For `SurfaceForcing`, `BoundaryForcing`, and `ROMSOutput`, chunking is performed **across time slices**.
* For `TidalForcing`, chunking is done **across tidal constituents**.

However, `InitialConditions` doesn't inherently have these natural chunking dimensions. Nevertheless, **parallelization is still necessary** due to the **memory-intensive operation of vertical regridding**, which involves creating several four-dimensional matrices of the size:
```
len(eta_rho) x len(xi_rho) x len(s_rho) x len(depth)
len(eta_v) x len(xi_rho) x len(s_rho) x len(depth)
len(eta_rho) x len(xi_u) x len(s_rho) x len(depth)
```
Here, the first three dimensions represent the ROMS target data, and the fourth dimension corresponds to the depth dimension of the source data.

To handle the memory load efficiently during vertical regridding, `ROMS-Tools` applies **horizontal chunking**, allowing vertical regridding to be performed in parallel. This parallelization is controlled by the `horizontal_chunk_size` parameter in `InitialConditions`, a unique feature not found in the other classes. (*Note: The `horizontal_chunk_size` parameter is only relevant if you set `use_dask = True`*). 

The default value for the `horizontal_chunk_size` parameter is **50**, and you generally don't need to change it (or specify it), unless:

* You want to experiment with performance optimization.
* You encounter memory blow-ups.
  
The optimal value for `horizontal_chunk_size` depends on your grid size of your grid, the size of your source data, and the available computational resources. Generally, the following guidelines should apply:

* **Increasing** `horizontal_chunk_size` (>50) → Faster computations, but higher memory usage.
* **Decreasing** `horizontal_chunk_size` (<50) → Lower memory usage, but slower computations.
  
To optimize for speed while managing memory, you can increase the `horizontal_chunk_size` until you reach your system's memory limit. 

Let’s now experiment with the `horizontal_chunk_size` to see how it impacts performance.

In [15]:
initial_conditions_with_dask_and_smaller_horizontal_chunks = InitialConditions(
    **initial_conditions_parameters,
    use_dask=True,
    horizontal_chunk_size=10
)

INFO - Sea surface height will NOT be used to adjust depth coordinates.
INFO - Selected time entry closest to the specified start_time (2012-01-02 00:00:00) within the range [2012-01-02 00:00:00, 2012-01-03 00:00:00]: ['2012-01-02T12:00:00.000000000']


In [16]:
%%time 
%%memit

initial_conditions_with_dask_and_smaller_horizontal_chunks.save(filename)

INFO - Writing the following NetCDF files:
/glade/derecho/scratch/noraloose/forcing/my_initial_conditions.nc


[########################################] | 100% Completed | 11m 3ss
peak memory: 29977.24 MiB, increment: 19099.20 MiB
CPU times: user 21min 21s, sys: 5min 57s, total: 27min 18s
Wall time: 12min 51s


Using a `horizontal_chunk_size` of 10 slowed down computations, but decreased the memory footprint.

In [19]:
initial_conditions_with_dask_and_larger_horizontal_chunks = InitialConditions(
    **initial_conditions_parameters,
    use_dask=True,
    horizontal_chunk_size=100
)

INFO - Sea surface height will NOT be used to adjust depth coordinates.
INFO - Selected time entry closest to the specified start_time (2012-01-02 00:00:00) within the range [2012-01-02 00:00:00, 2012-01-03 00:00:00]: ['2012-01-02T12:00:00.000000000']


In [20]:
%%time 
%%memit

initial_conditions_with_dask_and_larger_horizontal_chunks.save(filename)

INFO - Writing the following NetCDF files:
/glade/derecho/scratch/noraloose/forcing/my_initial_conditions.nc


[########################################] | 100% Completed | 130.50 s
peak memory: 38724.47 MiB, increment: 19651.17 MiB
CPU times: user 13min 9s, sys: 6min 43s, total: 19min 53s
Wall time: 2min 15s


Using a `horizontal_chunk_size` of 200 sped up computations, but increased the memory footprint.

## When To Use `Dask`?
As a general rule, we recommend enabling `Dask` (`use_dask = True`) for all five classes in `ROMS-Tools` that integrate with Dask (listed at the top of this notebook), unless you're working with very small test domains. 

<div class="alert alert-info">

Note

For tidal forcing (`TidalForcing`) you may not see a significant benefit from using `Dask`, as the computations are generally fast and have a low memory footprint. In such cases, using `Dask` might introduce more overhead than it provides benefit. However, the performance difference is usually minimal (we're talking a few seconds).
    
</div>

Below, we highlight three scenarios where enabling `Dask` is crucial—these cover the majority of use cases.

### 1. Large Input Datasets

Working with large input datasets (e.g., ERA5 or GLORYS) can lead to high memory usage if `use_dask = False`, particularly when dealing with global-scale data **spanning many time slices (months to years)**. This is especially relevant for `SurfaceForcing` and `BoundaryForcing`, where many time slices may need to be loaded at once. For reference, here’s an example of the storage requirements for one year of ERA5 and GLORYS data, even when restricted to the North Atlantic:

In [ ]:
!du -hs /global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/
!du -hs /global/cfs/projectdirs/m4746/Datasets/GLORYS/NA/2012/

### 2. Large Output Datasets
In some cases, the dataset stored in the `.ds` attribute is inherently large and may not fit into memory. This is particularly relevant for `SurfaceForcing` and `BoundaryForcing`, where the processed dataset can be extensive due to long time periods and high-resolution grids. When working with large output datasets, enabling Dask (`use_dask = True`) is crucial to avoid memory overflow. Below is an example of how much memory the output dataset requires for one year of ERA5 data on a medium-sized grid.

In [16]:
from roms_tools import SurfaceForcing

surface_forcing = SurfaceForcing(
    grid=Grid(nx=1000, ny=1000, size_x=1000, size_y=1000, center_lon=-20, center_lat=65, rot=0),
    start_time=datetime(2012, 1, 1),
    end_time=datetime(2012, 12, 31),
    source={"name": "ERA5", "path": "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5*.nc"},
    use_dask=True,
)

print(f"Memory footprint of output dataset: {surface_forcing.ds.nbytes / 1e6:.2f} MB")

INFO - Data will be interpolated onto grid coarsened by factor 2.


Memory footprint of output dataset: 61818.74 MB


Note that the output dataset requires more memory than the input dataset (which was 11GB). This can happen when working with high-resolution grids, where regridding increase the dataset size beyond the original input.

### 3. Memory-Intensive Intermediate Computations
Even if neither large input datasets nor large output datasets are an issue, certain operations can still be highly memory-intensive. This is particularly true for `InitialConditions`, where vertical regridding requires substantial memory. In our example above, `InitialConditions` could be processed with `use_dask = False`, leading to a peak memory usage of 46GB. However, as the grid size (`nx` and `n`y) increases, memory demands will scale accordingly—eventually exceeding available system memory and causing failures.

## Different Dask deployment options

## Bypassing validation

By default, `ROMS-Tools` classes perform a validation step during initialization to ensure the source data adequately covers the grid, particularly in the horizontal direction. In practice, this validation step detects NaNs after regridding. 

You can disable validation by setting `bypass_validation = True` (default: `False`), but **this is not recommended**. Skipping validation means missing crucial checks, such as verifying full grid coverage and detecting NaNs—values that would otherwise be replaced with a fill value, potentially leading to incorrect model results.

The following classes support the `bypass_validation` parameter:
* `TidalForcing`
* `SurfaceForcing`
* `BoundaryForcing`
* `InitialConditions`
  

**Important Considerations**:
* If `use_dask = False`, skipping validation **does not** improve performance, as data is computed during initialization regardless. Setting `bypass_validation = True` offers no benefit.
* If `use_dask = True`, skipping validation has minimal impact **except** for BGC initial conditions (the example was covered above), where a full 3D NaN check of `ALK` is performed. If you are certain that your source datasets (e.g., global datasets) fully cover the domain, you may choose to set `bypass_validation = True`. However, `ALK` will still be computed at the `.plot()` or `.save()` stage, meaning you're only delaying the computation.

In [21]:
%%time 
%%memit

initial_conditions_with_dask_and_bypassed_validation = InitialConditions(
    **initial_conditions_parameters,
    use_dask=True,
    bypass_validation=True
)

INFO - Sea surface height will NOT be used to adjust depth coordinates.
INFO - Selected time entry closest to the specified start_time (2012-01-02 00:00:00) within the range [2012-01-02 00:00:00, 2012-01-03 00:00:00]: ['2012-01-02T12:00:00.000000000']


peak memory: 19337.31 MiB, increment: 354.21 MiB
CPU times: user 7.04 s, sys: 1.04 s, total: 8.08 s
Wall time: 8.31 s


Initialiation was quicker and required less memory. The `.ds` attribute holds truly lazy data, including `zeta` and `ALK`:

In [22]:
initial_conditions_with_dask_and_bypassed_validation.ds

<xarray.Dataset> Size: 4GB
Dimensions:      (ocean_time: 1, eta_rho: 502, xi_rho: 502, s_rho: 100,
                  xi_u: 501, eta_v: 501, s_w: 101)
Coordinates:
    abs_time     (ocean_time) datetime64[ns] 8B 2012-01-02T12:00:00
  * ocean_time   (ocean_time) float64 8B 3.788e+08
Dimensions without coordinates: eta_rho, xi_rho, s_rho, xi_u, eta_v, s_w
Data variables: (12/42)
    zeta         (ocean_time, eta_rho, xi_rho) float32 1MB dask.array<chunksize=(1, 502, 502), meta=np.ndarray>
    temp         (ocean_time, s_rho, eta_rho, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    salt         (ocean_time, s_rho, eta_rho, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    u            (ocean_time, s_rho, eta_rho, xi_u) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    v            (ocean_time, s_rho, eta_v, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    ubar         (ocean_time, eta_rho, xi_u) float32 1MB dask.array<chunksize=(1, 50, 50), meta=np.ndarray>
    ...           ...
    diazFe       (ocean_time, s_rho, eta_rho, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    spCaCO3      (ocean_time, s_rho, eta_rho, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    zooC         (ocean_time, s_rho, eta_rho, xi_rho) float32 101MB dask.array<chunksize=(1, 100, 50, 50), meta=np.ndarray>
    w            (ocean_time, s_w, eta_rho, xi_rho) float32 102MB 0.0 ... 0.0
    Cs_r         (s_rho) float32 400B -0.992 -0.9753 ... -8.89e-05 -9.874e-06
    Cs_w         (s_w) float32 404B -1.0 -0.9837 -0.9667 ... -3.95e-05 0.0
Attributes:
    title:                                ROMS initial conditions file create...
    roms_tools_version:                   0.1.dev165
    ini_time:                             2012-01-02 00:00:00
    model_reference_date:                 2000-01-01 00:00:00
    adjust_depth_for_sea_surface_height:  False
    source:                               GLORYS
    bgc_source:                           CESM_REGRIDDED
    theta_s:                              5.0
    theta_b:                              2.0
    hc:                                   300.0

Note that at the saving stage `.save()`, `zeta` and `ALK` will still need to be computed, so what you gained during initialization, you will lose during compute. Plus, you might have NaNs filled in with fill value that you can't recover.
In short, `bypass_validation = True` is **not recommended**, unless you're debugging or if you're absolutely certain your dataset fully covers the ROMS domain. Even then, a NaN check remains a useful sanity check.